<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/result/kobert_nsmc_0_8_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요 라이브러리 설치

In [ ]:
!pip install transformers datasets accelerate scikit-learn optuna sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install transformers

In [ ]:

import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch
import optuna
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import platform
import transformers
import joblib

In [ ]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# 사용할 데이터

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  'My Drive'   이전   졸업논문


In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_train_불균형_0.8.csv', encoding='utf-8-sig')

In [ ]:
df.head(10)

,Unnamed: 0.1,Unnamed: 0,id,document,label
0,0,29755,7546373,"제가 10점을 드리는 이유는, 흥행이 목적이 아닌 영화인게 첫째구요. 우리나라사람에...",1
1,1,80949,7696701,여러가지를 생각하게 하는 아주 좋은 작품 입니다.,1
2,2,43956,8716532,나는 일본을 싫어하지만 일본영화가 참 좋다. 평범하면서도 소소하고 우리 일상같은 느...,1
3,3,89480,753929,아랫님들 '몽둥이 액션'이란 말이 딱 어울리네요 ㅎㅎ,1
4,4,44102,4871315,애니메이션은 언제나 교훈과 재미를 준다,1
5,5,61805,8705971,7-8점은 될만한 영환데 평점이 낮아서 10.인간의 죄의식을 비밀로써 덜어줌.,1
6,6,89117,2192515,지옥의묵시록만큼위대하진않지만...리들리 스콧다운영화임.,1
7,7,40807,10157331,이 영화를 안 순간부터 여러분들은 무뚝뚝하고 무서운 사람들에게 터미네이터란 별명을 ...,1
8,8,34680,1356468,1점줄꺼면 아예 평가를하질마 졸라재밋네,1
9,9,80373,1595078,오즈의 영역을 넘어 마술의 경지에 이르는 허우.,1


In [ ]:
df['label'].value_counts()

,count
label,
1,5000
0,4000


In [ ]:
df_test=pd.read_excel('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_test.xlsx')

In [ ]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

In [ ]:
train.shape

(9000, 2)

In [ ]:
test.shape

(2000, 2)

# 데이터 정제

In [ ]:
def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)

In [ ]:
train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)


Cleaning Special Characters: 100%|██████████| 2000/2000 [00:00<00:00, 14512.79it/s]


In [ ]:
# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# KoBERT

In [ ]:
# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [ ]:
# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

In [ ]:
# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 최적의 하이퍼파라미터 찾기

In [ ]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# Objective 함수 정의
def objective(trial):
    # 하이퍼파라미터 샘플링
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-4, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.3)

    # TrainingArguments 설정
    training_args = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        logging_steps=10,
        per_device_train_batch_size=batch_size,
        num_train_epochs=10,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1",
        report_to="none"
    )

    # Trainer 생성
    trainer = Trainer(
        model_init=model_init,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_f1"]


# Optuna 탐색
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=5)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", study.best_params)

[I 2025-02-16 12:46:26,129] A new study created in memory with name: no-name-b180dec9-a46e-4a14-b32d-96e9cc4d9fbf
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.358800,0.384647,0.850484
2,0.369100,0.413959,0.835939
3,0.145900,0.539535,0.839815
4,0.080000,0.682747,0.834404
5,0.026400,0.680020,0.856968
6,0.043900,0.796308,0.848770
7,0.007500,0.946831,0.842999
8,0.000500,0.959667,0.853991
9,0.018500,1.018362,0.851941
10,0.003000,1.049351,0.849410


[I 2025-02-16 13:53:31,628] Trial 0 finished with value: 0.8569678177589958 and parameters: {'learning_rate': 2.72641901855272e-05, 'batch_size': 16, 'weight_decay': 0.25831718342927346}. Best is trial 0 with value: 0.8569678177589958.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be a

Epoch,Training Loss,Validation Loss,F1
1,0.365600,0.364571,0.846472
2,0.368900,0.373979,0.843324
3,0.228500,0.438554,0.850424
4,0.114600,0.535815,0.842984
5,0.056500,0.618442,0.848494
6,0.083300,0.686873,0.838915
7,0.130300,0.790871,0.833678
8,0.059600,0.792570,0.847996
9,0.065400,0.841787,0.843859
10,0.076800,0.886039,0.838194


[I 2025-02-16 15:00:20,252] Trial 1 finished with value: 0.8504242772903783 and parameters: {'learning_rate': 1.1074102905825164e-05, 'batch_size': 16, 'weight_decay': 0.0004638259957442714}. Best is trial 0 with value: 0.8569678177589958.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to 

...

Epoch,Training Loss,Validation Loss,F1
1,0.701800,0.699291,0.333333
2,0.698700,0.695424,0.333333
3,0.696800,0.695235,0.333333
4,0.686400,0.693573,0.333333
5,0.691300,0.712479,0.333333
6,0.700600,0.711074,0.333333
7,0.689100,0.712864,0.333333
8,0.691500,0.699963,0.333333
9,0.696700,0.697378,0.333333
10,0.690200,0.698629,0.333333


[I 2025-02-16 17:10:24,319] Trial 3 finished with value: 0.3333333333333333 and parameters: {'learning_rate': 0.0003436315755223001, 'batch_size': 64, 'weight_decay': 0.24297125796854893}. Best is trial 2 with value: 0.8619689430121777.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be 

Epoch,Training Loss,Validation Loss,F1
1,0.392500,0.399255,0.836541
2,0.263000,0.367407,0.850489
3,0.158400,0.466630,0.850437
4,0.088900,0.600141,0.839952
5,0.050400,0.619477,0.857456
6,0.048000,0.681841,0.853497
7,0.015300,0.733597,0.851388
8,0.018600,0.910255,0.849709
9,0.001500,0.881485,0.850387
10,0.002700,0.914887,0.849856


[I 2025-02-16 18:15:49,060] Trial 4 finished with value: 0.8574563460059643 and parameters: {'learning_rate': 5.1365479932066135e-05, 'batch_size': 32, 'weight_decay': 0.26607191548313003}. Best is trial 2 with value: 0.8619689430121777.


Best Hyperparameters: {'learning_rate': 2.3500050365653693e-05, 'batch_size': 32, 'weight_decay': 0.02002678712268746}


Best Hyperparameters: {'learning_rate': 2.3500050365653693e-05, 'batch_size': 32, 'weight_decay': 0.02002678712268746}

# 최적의 하이퍼파라미터로 재학습

In [ ]:
#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [ ]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Best Hyperparameters: {'learning_rate': 2.3500050365653693e-05, 'batch_size': 32, 'weight_decay': 0.02002678712268746}

In [ ]:
# 7. 최적의 파라미터로 학습 설정
# 최적의 하이퍼파라미터를 찾은 후, training_args를 설정합니다.
training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=32,  # 배치 크기 고정
    learning_rate=2.3500050365653693e-05,  # 최적화된 학습률
    weight_decay=0.02002678712268746,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    disable_tqdm=False,
    load_best_model_at_end=True,
    report_to="none"
)

# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

Epoch,Training Loss,Validation Loss,F1
1,0.378700,0.360616,0.849386
2,0.291100,0.418545,0.850907
3,0.154900,0.406242,0.852991
4,0.094500,0.540351,0.847768
5,0.080500,0.598419,0.851487
6,0.038900,0.687368,0.846823
7,0.018400,0.754224,0.852500
8,0.026200,0.828906,0.846458
9,0.014700,0.871858,0.840275
10,0.011800,0.861199,0.844950


Elapsed Time: 1077.78 seconds
Average CPU Usage: 14.60%


In [ ]:
# 모델 저장
joblib.dump(trainer7, '/content/gdrive/MyDrive/trainer_news_text.pkl')

print("모델이 저장되었습니다.")

모델이 저장되었습니다.


In [ ]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

loaded_model = joblib.load('/content/gdrive/MyDrive/trainer_news_text.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = loaded_model.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

In [ ]:
test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]
selected_columns7.head(10)

,text,category_prediction
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,폴리스스토리 시리즈는 뉴까지 버릴께 하나도 없음 최고,1
3,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,0
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
5,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,완전 감동입니다 다시봐도 감동,1
7,개들의 나오나요 나오면 보고 싶음,1
8,내 나이와 같은 영화를 지금 본 나는 감동적이다하지만 훗날 다시보면대사하나하나 그 ...,1
9,영화의 오페라화라고 해야할 작품 극단적 평갈림은 어쩔 수 없는 듯,0


In [ ]:
selected_columns7.to_csv('nsmc_0.8_prediction.csv', encoding='utf-8-sig')

# 평가

In [37]:
from sklearn.metrics import f1_score

import pandas as pd

df=pd.read_csv('/content/nsmc_0.8_prediction.csv', encoding='utf-8-sig')

df.head(3)

,Unnamed: 0,text,y_true,y_pred,Unnamed: 4
0,0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1,1,True
1,1,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1,1,True
2,2,폴리스스토리 시리즈는 뉴까지 버릴께 하나도 없음 최고,1,1,True


In [40]:
df.columns

Index(['Unnamed: 0', 'text', 'y_true', 'y_pred', 'Unnamed: 4'], dtype='object')

In [42]:
from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8698    0.8220    0.8452      1000
           1     0.8313    0.8770    0.8535      1000

    accuracy                         0.8495      2000
   macro avg     0.8506    0.8495    0.8494      2000
weighted avg     0.8506    0.8495    0.8494      2000

